## Using this notebook
 
- You have to execute this notebook by running each cell consecutively.
- To retain any changes made, you need to save the notebook. 

For detailed information about the notebook interface see _Help -> User Interface Tour_ in the menu, and the [Guidebook](https://lbologna.github.io/hbp-sp6-guidebook/getting_started/getting_started.html#working-with-collabs) for working with Collabs.

In [ ]:
import os
import re
import pytz
import numpy
import pandas
import shutil
import shutil
import plotly
import tarfile
import dateutil
import warnings
import requests
import functools
import subprocess
import unicodedata
import xml.etree.ElementTree

import readconffile
import readexpfile

from functools import partial
from ipywidgets import widgets
from IPython.display import HTML, Javascript, display, clear_output

from plotly.offline import iplot, plot
from plotly.graph_objs import graph_objs as go
from plotly.graph_objs import Scatter, Layout
plotly.offline.init_notebook_mode()

In [ ]:
%%capture
!pip install NEURON-nightly --upgrade
!rm -rf /opt/app-root/lib/nrn

In [ ]:
if os.path.isdir('GUISYNANAL')==True:
    shutil.rmtree('GUISYNANAL')
if os.path.isdir('GUISYNANAL')==False:
    !mkdir GUISYNANAL
os.chdir("GUISYNANAL")
for f in os.listdir(os.getcwd()):
    if f.endswith('.txt'): 
        os.remove(f)

os.mkdir("transfer")
shutil.copy2(os.path.join("..", "data", 'ProbGABAAB_EMS_GEPH_g.mod'), 'transfer')
shutil.copy2(os.path.join("..", "data", 'netstims.mod'), 'transfer')
shutil.copy2(os.path.join("..", "readexpfile.py"), 'transfer')
shutil.copy2(os.path.join("..", "readconffile.py"), 'transfer')
shutil.copy2(os.path.join("..", "fitting.py"), 'transfer')
shutil.copy2(os.path.join("..", "fitness.py"), 'transfer')
shutil.copy2(os.path.join("..", "cellprop.py"), 'transfer')


javascript2 = """
<script type="text/Javascript">
document.styleSheets[0].insertRule(".widget-listbox.form-control { width: auto }", 1);
document.styleSheets[0].insertRule(".form_login > label { width: 30% }", 1);
document.styleSheets[0].insertRule(".form_login > label + input { width: 70% }", 1);
</script>
"""

h1=HTML(javascript2)
h1.display='none'
display(h1)

In [ ]:
warnings.filterwarnings('ignore') 

In [ ]:
os.chdir("..")
%run "unicore_api_Python3.ipynb"
os.chdir("GUISYNANAL")

In [ ]:
# NSG

def checkloginNSG(_):
    username=username_widget.value
    password=password_widget.value
    KEY = 'Application_Fitting-DA5A3D2F8B9B4A5D964D4D2285A49C57'
    headers = {'cipres-appkey' : KEY}
    URL = 'https://nsgr.sdsc.edu:8443/cipresrest/v1/job/'+username
    r = requests.get(URL, auth=(username, password),headers=headers,verify=False)
    root = xml.etree.ElementTree.fromstring(r.text)
    a=0
    for child in root:
        if child.tag == 'displayMessage':
            a=1
            msg=child.text
    if a==0:
        msg='Authenticated successfully'
        msg_widget.disabled=True
        buttonlogin.disabled=True
        username_widget.disabled=True
        password_widget.disabled=True
        BTV.disabled=False
    msg_widget.value=msg
    msg_widget.layout.display=''

username_widget = widgets.Text(description='Username:')
username_widget.layout.width='249px'
password_widget = widgets.Password(description='Password:')
buttonlogin = widgets.Button()
buttonlogin.description = 'Login NSG'
buttonlogin.background_color='gainsboro'
buttonlogin.border_color='black'
buttonlogin.width='20%'
buttonlogin.on_click(checkloginNSG)
msg_widget = widgets.Text(layout=widgets.Layout(width='40%'))
msg_widget.layout.display='none'

global msg
msg='not connected'


def movetostorageNSG(job_url,jobstage,outputuri,name,BT):
    KEY = 'Application_Fitting-DA5A3D2F8B9B4A5D964D4D2285A49C57'
    headers = {'cipres-appkey' : KEY}
    username=username_widget.value
    password=password_widget.value
    CRA_USER = username
    PASSWORD = password
    if jobstage=='COMPLETED':
        BT.disabled=True
        r = requests.get(outputuri,headers=headers, auth=(CRA_USER, PASSWORD),verify=False)
        globaldownloadurilist = []
        lengths = []
        root = xml.etree.ElementTree.fromstring(r.text)
        for child in root:
            if child.tag == 'jobfiles':
                for jobchild in child:
                    if jobchild.tag == 'jobfile':
                        for downloadchild in jobchild:
                            if downloadchild.tag == 'downloadUri':
                                for attchild in downloadchild:
                                    if attchild.tag == 'url':
                                        globaldownloadurilist.append(attchild.text)
                            if downloadchild.tag == 'length':
                                lengths.append(downloadchild.text)
        storeto_path = os.path.join('resultsNSG')
        if not os.path.exists(storeto_path):
            os.mkdir(storeto_path)
        storeto = str(os.path.join(storeto_path, name))
        if not os.path.exists(storeto):
            os.mkdir(storeto)
            os.chdir(storeto)
            for downloaduri in globaldownloadurilist:
                if float(lengths[globaldownloadurilist.index(downloaduri)])<=5048000:
                    r = requests.get(downloaduri, auth=(CRA_USER, PASSWORD), headers=headers,verify=False)
                    d = r.headers['Content-Disposition']
                    filename_list = re.findall('filename=(.+)', d)
                    for filename in filename_list:
                        if filename=='output.tar.gz' or filename=='STDOUT' or filename=='STDERR':
                            with open(filename, 'wb') as fd:
                                for chunk in r.iter_content():
                                    fd.write(chunk)
                            if filename=='output.tar.gz':
                                ff = tarfile.open("output.tar.gz")
                                ff.extractall('OUTPUT')
                                ff.close() 
                                os.remove('output.tar.gz')
            os.chdir("../..")


def getjobsboxNSG(site):
    ALERT.layout.display=''
    listbox=[]
    KEY = 'Application_Fitting-DA5A3D2F8B9B4A5D964D4D2285A49C57'
    headers = {'cipres-appkey' : KEY}
    username=username_widget.value
    password=password_widget.value
    URL = 'https://nsgr.sdsc.edu:8443/cipresrest/v1/job/'+username
    CRA_USER = username
    PASSWORD = password
    r_all = requests.get(URL, auth=(CRA_USER,PASSWORD), headers=headers,verify=False)
    root_all = xml.etree.ElementTree.fromstring(r_all.text)
    job_list = root_all.find('jobs')
    job_name_list=[]
    job_url_list=[]
    job_title_list=[]
    global job_submissiontime_list
    job_submissiontime_list=[]
    job_status_list=[]
    job_outputuri_list=[]
    for job in job_list.findall('jobstatus'):
        job_url=job.find('selfUri').find('url').text
        job_url_list.append(job_url)
        job_title=job_url[len(URL)+1:len(job_url)]
        job_title_list.append(job_title)
        r = requests.get(job_url, auth=(CRA_USER, PASSWORD), headers=headers,verify=False)
        root = xml.etree.ElementTree.fromstring(r.text)
        if root.find('metadata')!=None:
            job_name=root.find('metadata').find('entry').find('value').text
        else:
            job_name=''
        if job_name=='':
            job_name='NSG_Job'
        job_name_list.append(job_name)
        job_submissiontime=root.find('dateSubmitted').text
        job_submissiontime_list.append(job_submissiontime)
        for child in root:
            if child.tag == 'jobStage':
                jobstage=child.text
            if child.tag == 'resultsUri':
                for urlchild in child:
                    if urlchild.tag == 'url':
                        outputuri = urlchild.text
        job_status_list.append(jobstage)
        job_outputuri_list.append(outputuri)  
    indexes=sorted(range(len(job_submissiontime_list)),key=job_submissiontime_list.__getitem__)
    items=[]
    text = widgets.Label()
    text.value='Name/ID'
    text.layout.width='500px'
    items.append(text)
    for i in range(len(job_status_list)):
        vart=job_name_list[indexes[i]]+'/'+job_title_list[indexes[i]][job_title_list[indexes[i]].find('NEURON77_TG')\
                                                                      :len(job_title_list[indexes[i]])]
        crr_w = widgets.Label()
        crr_w.value=vart
        crr_w.layout.width='500px'
        if job_title_list[indexes[i]].find('NEURON77_TG')!=-1:
            items.append(crr_w)                                      
    box = widgets.VBox(items)
    listbox.append(box)
    items2=[]
    text = widgets.Label()
    text.value="Status"
    text.layout.width='180px'
    items2.append(text)
    for i in range(len(job_status_list)):
        if job_title_list[indexes[i]].find('NEURON77_TG')!=-1:
            text = widgets.Label()
            text.value=job_status_list[indexes[i]]
            text.layout.width='100px'
            sublist = [text]
            if job_status_list[indexes[i]]=='COMPLETED':
                file = open("../data/img/check.png", "rb")
                image = file.read()
                sublist.append(widgets.Image(value=image, format='png', width=25))
            items2.append(widgets.HBox(sublist))
    box2 = widgets.VBox(items2)
    listbox.append(box2)
    items3=[]
    text = widgets.Label()
    text.value="Submission Date"
    text.layout.width='180px'
    items3.append(text)
    for i in range(len(job_status_list)):
        if job_title_list[indexes[i]].find('NEURON77_TG')!=-1:
            text = widgets.Label()
            text.value=dateutil.parser.parse(job_submissiontime_list[indexes[i]]).astimezone(pytz.timezone('CET')).strftime("%d/%m/%Y %H:%M:%S")
            text.layout.width='180px'
            items3.append(text)
            
    box3 = widgets.VBox(items3)
    listbox.append(box3)
    items4=[]
    text = widgets.Label()
    text.value="Move to Storage"
    text.layout.width='180px'
    items4.append(text)
    for i in range(len(job_status_list)):
        if job_title_list[indexes[i]].find('NEURON77_TG')!=-1:
            BT=widgets.Button(description='Move')
            #foldername=job_name_list[indexes[i]]+'_'+job_title_list[indexes[i]]
            foldername=job_name_list[indexes[i]]+'_'+dateutil.parser.parse(job_submissiontime_list[indexes[i]]).\
            astimezone(pytz.timezone('CET')).strftime("%d-%m-%Y-%H:%M:%S")
            if not os.path.exists(str(os.path.join('resultsNSG',foldername))):
                BT.on_click(partial(movetostorageNSG,job_url_list[indexes[i]],job_status_list[indexes[i]],job_outputuri_list[indexes[i]],foldername))
            else:
                BT.disabled=True
            items4.append(BT)
    box4 = widgets.VBox(items4)
    listbox.append(box4)
    ALERT.layout.display='none'
    return listbox

In [ ]:
# NSG-SA

def move_to_storage_NSGSA(jobid, job_status, name, BT):
    DOWNLOAD_OUTPUT_URL = 'https://bspsa.cineca.it/files/nsg/bsp_nsg_01/' + jobid + '/'
    if job_status=='COMPLETED':      
        BT.disabled=True
        r = requests.get(url=DOWNLOAD_OUTPUT_URL, headers=get_oidc_auth())
        if r.status_code != 200:
            return
        files = json.loads(r.content)
        
        storeto_path = os.path.join('resultsNSG-SA')
        if not os.path.exists(storeto_path):
            os.mkdir(storeto_path)
        storeto = str(os.path.join(storeto_path, name))
        if not os.path.exists(storeto):
            os.mkdir(storeto)
        os.chdir(storeto)
        for f in files:
            r = requests.get(url=DOWNLOAD_OUTPUT_URL + str(f['fileid']) + '/', headers=get_oidc_auth())
            if r.status_code != 200:
                return
            if f['filename'] == 'output.tar.gz' or f['filename'] == 'STDOUT' or f['filename'] == 'STDERR':
                with open(f['filename'], 'wb') as fd:
                    for chunk in r.iter_content():
                        fd.write(chunk)
                    fd.close()
                if f['filename'] == 'output.tar.gz':
                    ff = tarfile.open(f['filename'])
                    ff.extractall('OUTPUT')
                    ff.close()
                    os.remove(f['filename'])
        os.chdir("../..")

    
def get_jobs_box_NSGSA():
    ALERT.layout.display=''
    
    ROOT_URL = 'https://bspsa.cineca.it/'
    JOBS_URL = ROOT_URL + 'jobs/nsg/bsp_nsg_01/'
    
    r = requests.get(url=JOBS_URL, headers=get_oidc_auth())
    
    if r.status_code != 200:
        return
    
    jobs = r.json()
        
    items = []
    text = widgets.Label()
    text.value='Name/ID'
    text.layout.width='500px'
    items.append(text)
    
    items2 = []
    text = widgets.Label()
    text.value="Status"
    text.layout.width='180px'
    items2.append(text)
    
    items3 = []
    text = widgets.Label()
    text.value="Submission Date"
    text.layout.width='180px'
    items3.append(text)
    
    items4 = []
    text = widgets.Label()
    text.value="Move to Storage"
    text.layout.width='180px'
    items4.append(text)        
    
    for job in jobs:
        if job['job_id'].find('NEURON77_TG')!=-1:
            job_title = job['title']
            if job_title == "":
                job_name = job['job_id']
            else:
                job_name = job_title + "/" +job['job_id'][job['job_id'].find('NEURON77_TG'):len(job['job_id'])]
            crr_w = widgets.Label()
            crr_w.value=job_name
            crr_w.layout.width='500px'
            items.append(crr_w)  
            text = widgets.Label()
            text.value=job['stage']
            text.layout.width='100px'
            sublist = [text]
            if job['stage']=='COMPLETED':
                file = open("../data/img/check.png", "rb")
                image = file.read()
                sublist.append(widgets.Image(value=image, format='png', width=25))  
            items2.append(widgets.HBox(sublist))
            text = widgets.Label()
            text.value=dateutil.parser.parse(job['init_date']).astimezone (pytz.timezone('CET')).strftime("%d/%m/%Y %H:%M:%S")
            text.layout.width='180px'
            items3.append(text)
        
            BT = widgets.Button(description='Move')
            foldername=job_title + "_" + dateutil.parser.parse(job['init_date']).astimezone(pytz.timezone('CET')).\
            strftime("%d-%m-%Y-%H:%M:%S")
            BT.on_click(partial(move_to_storage_NSGSA,job['job_id'], job['stage'], foldername))
            #foldername = job['job_id']
        
            if job['stage'] != 'COMPLETED':
                BT.tooltip = 'Still waiting'
                BT.disabled = True
        
            if os.path.exists(str(os.path.join('resultsNSG-SA', foldername))):
                BT.tooltip = 'Already moved'
                BT.disabled = True
            
            items4.append(BT)
    
    listbox = [widgets.VBox(items), widgets.VBox(items2), widgets.VBox(items3), widgets.VBox(items4)]  
    ALERT.layout.display='none'
    return listbox

In [ ]:
def getjobsboxB(site):
    B.visible=False
    B.children=[]
    if site=='NSG':                                               #
        listbox=getjobsboxNSG(site)                               #
    elif site == 'Service Account - NSG':                         # SERVICE ACCOUNT
        listbox = get_jobs_box_NSGSA()                            #
    box_layout = widgets.Layout(display='flex',flex_flow='row')
    B.layout=box_layout
    B.children=listbox
    B.visible=True

def changesystem(_):
    listbox=[]
    B.children=listbox
    BTV.disabled=False
    BTR.layout.display='none'
    if S.value=='NSG':
        BTV.disabled=True
        formMNSG.layout.display=''
    else:
        formMNSG.layout.display='none'
        
def viewsym(_):
    refresh(_)
    BTR.layout.display=''
    
def refresh(_):
    BTR.disabled=True
    BTV.disabled=True
    S.disabled=True
    if S.value=='NSG' or 'Service Account - NSG':                 # SERVICE ACCOUNT
        getjobsboxB(S.value) 
    BTR.disabled=False
    S.disabled=False
    
RB11=widgets.RadioButtons(description= 'Select data file', options=['example','storage'],\
                         style={'description_width': 'initial'})
DM1=widgets.Text()
DM1.value='testB43100.csv'
DM1.layout.display=''
DM1.disabled=True
DM2=widgets.Text()
DM2.value='no results in the collab storage'
DM2.layout.display='none'
DM2.disabled=True

dirloc=os.getcwd()
SF=widgets.SelectMultiple(description='folders', options=[],selected_labels=[]) 
SF.layout.width="75%"
SF.layout.display='none'
SFLabel = widgets.Label()
SFLabel.value = ''

In [ ]:
def getlistoffoldersandfoldername():
    foldername='foldername'
    if os.path.exists(os.path.join('resultsNSG')):
        if os.path.exists(os.path.join('resultsNSG', SF.value[0])):
            FIlist=os.listdir(os.path.join('resultsNSG', SF.value[0]))
            foldername='resultsNSG'
            return FIlist, foldername
    if os.path.exists(os.path.join('resultsNSG-SA')):
        if os.path.exists(os.path.join('resultsNSG-SA', SF.value[0])):
            FIlist =os.listdir(os.path.join('resultsNSG-SA', SF.value[0]))
            foldername = 'resultsNSG-SA'
            return FIlist, foldername
    


def outputfiles():
    global FIlist,FIlisttxt,FIlistcsv
    os.chdir("..")
    FIlist, foldername = getlistoffoldersandfoldername()
    os.chdir("GUISYNANAL")
    FIlisttxt=[]
    FIlistcsv=[]
    for file in FIlist:
        if file.endswith('.txt'):  
            FIlisttxt.append(file)
        if file.endswith('.csv'):  
            FIlistcsv.append(file)
    if len(FIlisttxt) == 0 and len(FIlistcsv) == 0:
        T.layout.display='none'
        BP.layout.display='none'
        BT.layout.display='none'
        SFLabel.value = 'No csv or txt file inside the folder!'
    else:
        T.layout.display=''
        BP.layout.display=''
        BT.layout.display=''
        SFLabel.value=''
        

def filesinfolder(change):
    with out1:
        clear_output()
    with out2:
        clear_output()
    with out3:
        clear_output()
    T.layout.display='none'
    BP.layout.display='none'
    BT.layout.display='none'
    if len(SF.value)==1:
        SFLabel.value = ''
        outputfiles()
    else:
        SFLabel.value = 'Select only one folder!'

In [ ]:
def RB11click(_):
    os.chdir("..")
    if RB11.value=='storage':
        with out1:
            clear_output()
        with out2:
            clear_output()
        with out3:
            clear_output()
        T.layout.display='none'
        BP.layout.display='none'
        BT.layout.display='none'
        DM1.layout.display='none'
        FOlist=[]
        for path in ["resultsNSG", "resultsNSG-SA"]:
            if os.path.exists(path):
                FOlist += [f[f.find('/')+1:len(f)] for f in os.listdir(path)]
        if len(FOlist)>0:
            SF.options = sorted(FOlist, key=str.lower);
            SF.observe(filesinfolder, names='value')
            SF.layout.display=''
            T.layout.display='none'
            BP.layout.display='none'
            BT.layout.display='none'
        else:
            DM2.layout.display=''
            T.layout.display='none'
            BP.layout.display='none'
            BT.layout.display='none'
    else:
        with out1:
            clear_output()
        with out2:
            clear_output()
        with out3:
            clear_output()
        DM1.layout.display=''
        DM2.layout.display='none'
        SF.layout.display='none'
        T.layout.display=''
        BP.layout.display=''
        BT.layout.display=''
    os.chdir("GUISYNANAL")

RB11.observe(RB11click,'value')

importedneuron=False
out1 = widgets.Output()
out2 = widgets.Output()
out3 = widgets.Output()

In [ ]:
def tabledata(_):
    with out1:
        clear_output()
    with out2:
        clear_output()
    with out3:
        clear_output()
    T.disabled=True
    BP.disabled=True
    BT.disabled=True
    tograph=0
    if RB11.value=='storage':
        os.chdir("..")
        FIlist, foldername = getlistoffoldersandfoldername()
        if foldername!='foldername':
            FL=os.listdir(str(os.path.join(foldername,SF.value[0])))
            fileconfig='fileconfig'
            for files in FL:
                if files.startswith('config'):
                    fileconfig=files
            if fileconfig!='fileconfig':
                if FIlistcsv[0]!='':
                    os.chdir(os.path.join(foldername, SF.value[0]))
                    fh=open(fileconfig,"r")
                    firstrow=fh.readline()
                    fh.close()
                    if firstrow.rstrip()=='//name of file containing raw traces':
                        readconffile.filename=fileconfig
                        [inputfilename,modfilename,parametersfilename,flagdata,flagcut,nrtraces,Vrestf,\
                         esynf,nrparamsfit,paramnr,paramname,paraminitval,paramsconstraints,nrdepnotfit,\
                         depnotfit,nrdepfit,depfit,seedinitvaluef]=readconffile.readconffile()
                        if os.path.exists(inputfilename.rstrip()):
                            NAMES=['trace','fitnr','error']
                            for k in range(len(paramname)):
                                NAMES.append(paramname[k])
                            NAMES.append('thresh')
                            NAMES.append('min')
                            COL=range(0,len(NAMES))
                            data=pandas.read_csv(FIlistcsv[0], sep='\t',usecols=COL,names=NAMES)
                            if data.shape[0] == 0:
                                SFLabel.value = 'Results file is empty. No fit was below threshold!'
                            else:
                                tograph=1
                        else:
                            SFLabel.value = 'Exp file corresponding to the config file is not in the storage!'
                    else:
                        SFLabel.value = 'This is not a config file!'
                    os.chdir("../..")
                else:
                    SFLabel.value = 'There is no results file in the folder!'
            else:
                SFLabel.value = 'There is no config file in the folder!'
        os.chdir("GUISYNANAL")
    else:
        os.chdir(os.path.join('..', 'data', 'config_files'))
        expf='expB4.txt'
        conff='configB4.txt'
        path = os.path.join('../..', 'GUISYNANAL', 'transfer/')
        for file in os.listdir(path):
            if file.endswith('.txt'): 
                os.remove(os.path.join(path, file))
        shutil.copy2(expf, path)
        shutil.copy2(conff, path)
        readconffile.filename=conff
        [inputfilename,modfilename,parametersfilename,flagdata,flagcut,nrtraces,Vrestf,esynf,nrparamsfit,\
         paramnr,paramname,paraminitval,paramsconstraints,nrdepnotfit,depnotfit,nrdepfit,depfit,\
         seedinitvaluef]=readconffile.readconffile()
        NAMES=['trace','fitnr','error']
        for k in range(len(paramname)):
            NAMES.append(paramname[k])
        NAMES.append('thresh')
        NAMES.append('min')
        COL=range(0,len(NAMES))
        os.chdir("..")
        data=pandas.read_csv('testB43100.csv', sep='\t',usecols=COL,names=NAMES)
        os.chdir('../GUISYNANAL')
        tograph=1
    if tograph==1:
        datasorted = data.sort_values([NAMES[2]], ascending=1)
        with out1:
            display(HTML(datasorted.to_html(index=False)))
    T.disabled=False
    BP.disabled=False
    BT.disabled=False

In [ ]:
def boxplot(_):
    with out1:
        clear_output()
    with out2:
        clear_output()
    with out3:
        clear_output()
    T.disabled=True
    BP.disabled=True
    BT.disabled=True
    boxi=[]
    tograph=0
    if RB11.value=='storage':
        os.chdir("..")
        FIlist, foldername = getlistoffoldersandfoldername()
        if foldername!='foldername':
            FL=os.listdir(str(os.path.join(foldername,SF.value[0])))
            fileconfig='fileconfig'
            for files in FL:
                if files.startswith('config'):
                    fileconfig=files
            if fileconfig!='fileconfig':
                if FIlistcsv[0]!='':
                    os.chdir(os.path.join(foldername, SF.value[0]))
                    fh=open(fileconfig,"r")
                    firstrow=fh.readline()
                    fh.close()
                    if firstrow.rstrip()=='//name of file containing raw traces':
                        readconffile.filename=fileconfig
                        [inputfilename,modfilename,parametersfilename,flagdata,flagcut,nrtraces,Vrestf,\
                         esynf,nrparamsfit,paramnr,paramname,paraminitval,paramsconstraints,nrdepnotfit,\
                         depnotfit,nrdepfit,depfit,seedinitvaluef]=readconffile.readconffile()
                        if os.path.exists(inputfilename.rstrip()):
                            NAMES=['trace','fitnr','error']
                            for k in range(len(paramname)):
                                NAMES.append(paramname[k])
                            NAMES.append('thresh')
                            NAMES.append('min')
                            COL=range(0,len(NAMES))
                            data=pandas.read_csv(FIlistcsv[0], sep='\t',usecols=COL,names=NAMES)
                            if data.empty==True:
                                SFLabel.value = "Results file is empty. No fitting was within threshold!"
                            else:
                                tograph=1
                        else:
                            SFLabel.value = "Exp file corresponding to the config file is not in the storage!"
                            os.chdir('..')
                            tograph=0
                    else:
                        SFLabel.value = "This is not a config file!"
                    os.chdir("../..")
                else:
                    SFLabel.value = "There is no results file in the folder!"
            else:
                SFLabel.value = "There is no config file in the folder!"
        os.chdir("GUISYNANAL")
    else:
        os.chdir(os.path.join('..', 'data', 'config_files'))
        expf='expB4.txt'
        conff='configB4.txt'
        path = os.path.join('../..', 'GUISYNANAL', 'transfer/')
        for file in os.listdir(path):
            if file.endswith('.txt'): 
                os.remove(os.path.join(path, file))
        shutil.copy2(expf, path)
        shutil.copy2(conff, path)
        readconffile.filename=conff
        [inputfilename,modfilename,parametersfilename,flagdata,flagcut,nrtraces,Vrestf,esynf,nrparamsfit,\
         paramnr,paramname,paraminitval,paramsconstraints,nrdepnotfit,depnotfit,nrdepfit,depfit,\
         seedinitvaluef]=readconffile.readconffile()
        NAMES=['trace','fitnr','error']
        for k in range(len(paramname)):
            NAMES.append(paramname[k])
        NAMES.append('thresh')
        NAMES.append('min')
        COL=range(0,len(NAMES))
        data=pandas.read_csv('../testB43100.csv', sep='\t',usecols=COL,names=NAMES)
        os.chdir('../../GUISYNANAL')
        tograph=1
    if tograph==1:
        with out2:
            data2=data
            for k in range(3,(len(NAMES)-2)):
                data2[NAMES[k]]=(data2[NAMES[k]]-numpy.mean(data2[NAMES[k]]))/numpy.std(data2[NAMES[k]])
            for k in range(3,(len(NAMES)-2)):
                boxi.append(go.Box(y = data2[NAMES[k]] ,name=NAMES[k],showlegend=True))
            layout = go.Layout(
                yaxis=dict(
                    hoverformat = '.2f'
                )
            )
            fig = go.Figure(data=boxi, layout=layout)
            iplot(fig)
    T.disabled=False
    BP.disabled=False
    BT.disabled=False

In [ ]:
def plotbest(data,NAMES,indexplot,vecparamsf,esynf,Vrestf,nrparamsfit,nrdepnotfit,modfilename,paramname,\
             depnotfit,nrdepfit,depfit,paramsconstraints):

    myvar = !nrnivmodl
    
    import neuron
    from neuron import h
    global importedneuron
    importedneuron=True
    
    import fitness
    import cellprop
    
    if RB11.value=='storage':
        fitness.filename3 = modfilename.rstrip()
    else:
        fitness.filename3='ProbGABAAB_EMS_GEPH_g.mod'  
    
    [sizeofsw,maxofsw,vec5,timevecS,cutsin]=fitness.finaltrace(trace_number=data[NAMES[0]][indexplot]);
    parameters=vecparamsf;
    
    tstop = 100
    e_syn = esynf
    Vrest = Vrestf
    netstim = neuron.h.NetStims(0.5, sec=cellprop.soma)
    netstim.freqhz = 18.0
    netstim.q = 0.0
    netstim.prob = 2.0
    netstim.noise = 1.0
    netstim.number = 1.0
    
    vclamp = neuron.h.VClamp(0.5, sec=cellprop.soma)
    vclamp.dur[0] = tstop
    vclamp.amp[0] = Vrest
    
    with open(fitness.filename3) as ff:
        searchlines=ff.readlines()
    for kk, line in enumerate(searchlines):
        if "POINT_PROCESS" in line:
            break
    l2=line.split()
    synapse = neuron.h.__getattribute__(l2[1])(.5)
    if RB11.value=='example' or fitness.filename3=='ProbGABAAB_EMS_GEPH_g.mod':
        synapse.verboseLevel = 0
        synapse.Use = 1.0
        synapse.u0 = 1.0
        synapse.e_GABAA = e_syn
        synapse.setRNG(cellprop.synapse_rng)

    netcon = neuron.h.NetCon(netstim, synapse )
    netcon.delay = 0.0
    netcon.threshold = 0.0
    
    vclamp_i = neuron.h.Vector()
    timevec = neuron.h.Vector()
    timevec.from_python(timevecS)

    neuron.h('''nrparamsfit=0''')
    neuron.h.nrparamsfit=nrparamsfit
    neuron.h('''objref paramnamenrn[nrparamsfit]''')
    for i in range(nrparamsfit):
        neuron.h.paramnamenrn[i]=neuron.h.String()
        neuron.h.paramnamenrn[i].s=paramname[i]   
    neuron.h('''objref parametersnrn''')
    neuron.h('''parametersnrn =new Vector()''')
    neuron.h.parametersnrn.from_python(parameters)
    for i in range(nrparamsfit):
        neuron.h('strdef cmdstr')
        neuron.h('a=0')
        neuron.h.a=i
        neuron.h.execute('sprint(cmdstr,"%s = %g", paramnamenrn[a].s, parametersnrn.x[a])')
        exec(neuron.h.cmdstr)
    for i in range(nrdepnotfit):
        cmd=depnotfit[i]
        exec(cmd)

    neuron.h.tstop = tstop
    
    exc=0
    for i in range(nrdepfit):
        cmd=depfit[i]
        if eval(cmd):
            exc=exc or 1
    
    paramnr=0
    for row in paramsconstraints:
        low=row[0]
        high=row[1]
        if (parameters[paramnr]<low or parameters[paramnr]>high):
            exc=exc or 1      
        paramnr=paramnr+1
    
    vclamp_i.record(synapse._ref_i, timevec)
    
    neuron.h.run()
    
    vclamp_i.mul(1000.0)
    
    with out3:
        model_current = vclamp_i.to_python() 
        layout = go.Layout(
            xaxis=dict(
                title='time (ms)',
                titlefont=dict(
                    family='Courier New, monospace',
                    size=18,
                    color='#7f7f7f'
                )
            ),
            yaxis=dict(
                title='current (pA)',
                    titlefont=dict(
                    family='Courier New, monospace',
                    size=18,
                    color='#7f7f7f'
                ),
                hoverformat = '.2f'
            )
        )
        trace1 = go.Scatter(x=timevecS,y=vec5,mode = 'lines', name = 'original')
        trace2 = go.Scatter(x=timevecS,y=model_current,mode = 'lines', name = 'fitted')
        iplot(go.Figure(data=[trace1,trace2], layout=layout))
    errorverif=0
    for k in range(len(timevec)):
        errorverif=errorverif+(model_current[k]-vec5[k])*(model_current[k]-vec5[k])
    errorverif=errorverif/len(vec5)

In [ ]:
def bestfit(_):
    with out1:
        clear_output()
    with out2:
        clear_output()
    with out3:
        clear_output()
    global importedneuron
    T.disabled=True
    BP.disabled=True
    BT.disabled=True        
    tograph=0
    if RB11.value=='storage':
        os.chdir("..")
        FIlist, foldername =getlistoffoldersandfoldername()
        if foldername!='foldername':
            FL=os.listdir(str(os.path.join(foldername,SF.value[0])))
            fileconfig='fileconfig'
            for files in FL:
                if files.startswith('config'):
                    fileconfig=files
            if fileconfig!='fileconfig':
                if FIlistcsv[0]!='':
                    for files in FL:
                        if files.endswith('.mod') and not(files.startswith('netstims')):
                            modfilename=files.rstrip()
                    if modfilename!='':
                        os.chdir(os.path.join(foldername,SF.value[0]))
                        
                        for f in os.listdir("../../GUISYNANAL/transfer"):
                            if f.endswith('.txt'): 
                                os.remove(os.path.join("../../GUISYNANAL/transfer", f))
                        
                        for f in os.listdir("."):
                            if f.endswith('.txt') or f.endswith('.mod'):
                                shutil.copy2(f, "../../GUISYNANAL/transfer")
                        
                        fh=open(fileconfig,"r")   
                        firstrow=fh.readline()
                        fh.close()
                        if firstrow.rstrip()=='//name of file containing raw traces':
                            readconffile.filename=fileconfig
                            [inputfilename,modfilename,parametersfilename,flagdata,flagcut,nrtraces,Vrestf,esynf,nrparamsfit,\
                            paramnr,paramname,paraminitval,paramsconstraints,nrdepnotfit,depnotfit,nrdepfit,\
                            depfit,seedinitvaluef]=readconffile.readconffile()
                            if os.path.exists(inputfilename.rstrip()):
                                readexpfile.filename2=inputfilename.rstrip()
                                NAMES=['trace','fitnr','error']
                                for k in range(len(paramname)):
                                    NAMES.append(paramname[k])
                                NAMES.append('thresh')
                                NAMES.append('min')
                                COL=range(0,len(NAMES))
                                data=pandas.read_csv(FIlistcsv[0], sep='\t',usecols=COL,names=NAMES)
                                if data.empty==True:
                                    SFLabel.value = "Results file is empty. No fitting was within threshold!"
                                else:
                                    tograph=1
                            else:
                                SFLabel.value = "Exp file corresponding to the config file is not in the storage!"
                        else:
                            SFLabel.value = "This is not a config file!"
                        os.chdir("../..")
                    else:
                        SFLabel.value = "This is no mod file in the folder!"
                else:
                    SFLabel.value = "There is no results file in the folder!"
            else:
                SFLabel.value = "There is no config file in the folder!"
        os.chdir("GUISYNANAL")
    else:
        os.chdir(os.path.join('..', 'data', 'config_files'))  
        expf='expB4.txt'
        conff='configB4.txt'
        path = os.chdir(os.path.join('../..', 'GUISYNANAL'))
        os.chdir("transfer")
        for f in os.listdir(os.getcwd()):
            if f.endswith('.txt'): 
                os.remove(f)
        shutil.copy2(os.path.join("..", "..", "data", 'config_files', conff), '.')
        shutil.copy2(os.path.join("..", "..", "data", 'config_files', expf), '.')
        

        readconffile.filename=conff
        readexpfile.filename2=expf
        
        [inputfilename,modfilename,parametersfilename,flagdata,flagcut,nrtraces,Vrestf,esynf,nrparamsfit,\
        paramnr,paramname,paraminitval,paramsconstraints,nrdepnotfit,depnotfit,nrdepfit,\
        depfit,seedinitvaluef]=readconffile.readconffile()
        
        NAMES=['trace','fitnr','error']
        for k in range(len(paramname)):
            NAMES.append(paramname[k])
        NAMES.append('thresh')
        NAMES.append('min')
        COL=range(0,len(NAMES))
        os.chdir("..")
        data=pandas.read_csv('../data/testB43100.csv', sep='\t',usecols=COL,names=NAMES)
        tograph=1

    if tograph==1:
        if not importedneuron:
            indexplot=[n for n,i in enumerate(data[NAMES[2]]) if i==min(data[NAMES[2]]) ][0]
            vecparamsf=[]
            for k in range(len(paramname)):
                vecparamsf.append(data[NAMES[k+3]][indexplot])
            os.chdir("transfer")
            if os.path.exists("x86_64"):
                shutil.rmtree("x86_64")
            plotbest(data,NAMES,indexplot,vecparamsf,esynf,Vrestf,nrparamsfit,nrdepnotfit,modfilename,\
                     paramname,depnotfit,nrdepfit,depfit,paramsconstraints)
            os.chdir('..')
        else:
            if RB11.value=='storage':
                modname=modfilename.rstrip()
            else:
                modname='ProbGABAAB_EMS_GEPH_g.mod'
            modcompiled = False
            os.chdir("transfer")
            for f in os.listdir(os.getcwd()):
                if f == modname:
                    modcompiled = True
            if modcompiled:
                indexplot=[n for n,i in enumerate(data[NAMES[2]]) if i==min(data[NAMES[2]]) ][0]
                vecparamsf=[]
                for k in range(len(paramname)):
                    vecparamsf.append(data[NAMES[k+3]][indexplot])
                plotbest(data,NAMES,indexplot,vecparamsf,esynf,Vrestf,nrparamsfit,nrdepnotfit,modfilename,\
                         paramname,depnotfit,nrdepfit,depfit,paramsconstraints)
            else:
                SFLabel.value = "Mod file is different. Neuron cannot load dll file again. Please restart kernel to plot the best fit!"
            os.chdir('..')
    T.disabled=False
    BP.disabled=False
    BT.disabled=False

In [ ]:
T = widgets.Button()
T.description = 'Results table'
T.background_color='gainsboro'
T.border_color='black'
T.width='20%'
T.on_click(tabledata)

BP = widgets.Button()
BP.description = 'Boxplot results'
BP.background_color='gainsboro'
BP.border_color='black'
BP.width='20%'
BP.on_click(boxplot)

BT = widgets.Button()
BT.description = 'Best fit'
BT.background_color='gainsboro'
BT.border_color='black'
BT.width='20%'
BT.on_click(bestfit)

buttonS2 =[T, BP, BT]
mybox2=widgets.HBox(children=buttonS2)
    
global B
box_layout = widgets.Layout(display='flex',flex_flow='row')
B=widgets.Box(children=[],layout=box_layout)
S=widgets.Dropdown(
    options=['NSG', 'Service Account - NSG'],
    description='HPC:',
    disabled=False
)

BTV = widgets.Button()
BTV.description= 'View simulations'
BTV.height="40px"
BTV.width="120px"
BTV.on_click(viewsym)
BTV.disabled = True
BTR = widgets.Button()
BTR.tooltip='Reload'
BTR.icon="fa-refresh"
BTR.height="40px"
BTR.width="60px"
BTR.on_click(refresh)
BTR.layout.display='none'
system= widgets.HBox([S,BTV,BTR])
S.observe(changesystem,'value')
ALERT=widgets.HTML("""<div class="warn">Fetching job list....</div>""")
ALERT.layout.display='none'
formMNSG=widgets.VBox(children=[username_widget,password_widget,buttonlogin,msg_widget])
formMNSG.layout.display=''
ALERT1=widgets.HTML("""<div class="warn">To view analyses already saved in this Collab's storage skip the 'View simulations' step.</div>""")
display(ALERT1)
accordion = widgets.Accordion(children=[widgets.VBox([system,ALERT,formMNSG,B]), \
                                        widgets.VBox([RB11,DM1,DM2,SF,SFLabel,mybox2,out1,out2,out3])])
accordion.set_title(0, 'View simulations')
accordion.set_title(1, 'View analysis')
accordion

In [ ]:
from IPython.display import HTML

HTML('''
<button class="toggle_button" onclick="toggleCode(this)">Show Code</button>
<script>
    function toggleCode(button) {
        boxes = button.closest(".jp-Notebook").querySelectorAll(".jp-Cell-inputWrapper");
        if (boxes[0].style.display == "none") {
            boxes.forEach(box => box.style.display = '');
            button.innerHTML = "Hide Code";
        } else {
            boxes.forEach(box => box.style.display = 'none');
            button.innerHTML = "Show Code";
        }
    }
    document.querySelectorAll(".jp-Cell-inputWrapper").forEach(box => box.style.display = 'none');
</script>
''')